In [ ]:
def get_optimizer_instance(opt, learning_rate=None):
"""Returns an optimizer instance.
Supports the following types for the given `opt`:
    * An `Optimizer` instance string: Returns the given `opt`.
    * A supported string: Creates an `Optimizer` subclass with the given `learning_rate`.
  Supported strings:
    * 'Adagrad': Returns an `AdagradOptimizer`.
    * 'Adam': Returns an `AdamOptimizer`.
    * 'Ftrl': Returns an `FtrlOptimizer`.
    * 'RMSProp': Returns an `RMSPropOptimizer`.
    * 'SGD': Returns a `GradientDescentOptimizer`.
Args:
  opt: An `Optimizer` instance, or supported string, as discussed above.
  learning_rate: A float. Only used if `opt` is a supported string.
Returns:
  An `Optimizer` instance.
Raises:
  ValueError: If `opt` is an unsupported string.
  ValueError: If `opt` is a supported string but `learning_rate` was not specified.
  ValueError: If `opt` is none of the above types.
"""
    # Methods related to optimizers used in canned_estimators."""
    _OPTIMIZER_CLS_NAMES = {
        'adagrad': tf.optimizers.Adagrad,
        'adam': tf.optimizers.Adam,
        'rmsprop': tf.optimizers.RMSprop,
        'sgd': tf.optimizers.SGD
    }
    if isinstance(opt, str):
        print(opt)
        if opt in _OPTIMIZER_CLS_NAMES:
            print(opt)
            if learning_rate is None:
                print(learning_rate)
                raise ValueError('learning_rate must be specified when opt is supported string.')
            return _OPTIMIZER_CLS_NAMES[opt](learning_rate=learning_rate)
        else:
            raise ValueError(
                'Unsupported optimizer option: `{}`. '
                'Supported names are: {} or an `Optimizer` instance.'.format(
                    opt, tuple(sorted(_OPTIMIZER_CLS_NAMES))))
    else:
        try:
            opt = eval(opt)  # eval('tf.train.MomentumOptimizer(0.1, 0.9)') tf.train.MomentumOptimizer(0.1, 0.9)
            if not isinstance(opt, tf.optimizers.Optimizer):
                raise ValueError('The given object is not an Optimizer instance. Given: {}'.format(opt))
            return opt
        except (AttributeError, NameError):
            raise ValueError(
                'Unsupported optimizer option: `{}`. '
                'Supported names are: {} or an `Optimizer` instance.'.format(
                    opt, tuple(sorted(_OPTIMIZER_CLS_NAMES))))

In [ ]:
class BaseModel(object):
    """Abstract base class."""

    def __init__(self, iterator, params):
        """Initialize model, build graph.
        Args:
          iterator: instance of class BatchedInput, defined in dataset.
          params: parameters.
        """
        self.pred = None
        self.merge = None
        self.train_op = None
        self.global_step = None
        self.loss = None
        self.predict = None
        self.t_neg = None
        self.h_neg = None
        self.r = None
        self.t = None
        self.h = None
        self.writer = tf.summary.create_file_writer("/tmp/graph_log")
        self.relation_embedding = None
        self.entity_embedding = None
        self.iterator = iterator
        self.params = params
        self.k = params.entity_embedding_dim  # entity embedding dimension
        self.d = self.k  # relation embedding dimension, default equal to entity embedding dimension
        self.b = params.batch_size  # batch_size
        self.scope = self.__class__.__name__  # instance class name


    def build_graph(self):
        """Call this method to build graph."""
        with tf.name_scope(self.scope):
            # embedding
            bound = 6 / math.sqrt(self.k)
            with tf.name_scope('embedding'):
                self.entity_embedding = tf.get_current_name_scope(
                    name='entity',
                    shape=[self.params.entity_size, self.k],
                    initializer=tf.random_uniform_initializer(-bound, bound))
                self.entity_embedding = tf.nn.l2_normalize(self.entity_embedding, axis=1)
                tf.summary.histogram(name=self.entity_embedding.op.name, data=self.entity_embedding)
                self.relation_embedding = tf.get_current_name_scope(
                    name='relation',
                    shape=[self.params.relation_size, self.d],
                    initializer=tf.random_uniform_initializer(-bound, bound))
                self.relation_embedding = tf.nn.l2_normalize(self.relation_embedding, axis=1)
                tf.summary.histogram(name=self.relation_embedding.op.name, data=self.relation_embedding)

            with tf.name_scope('lookup'):
                self.h = tf.nn.embedding_lookup(self.entity_embedding, self.iterator.h)
                self.t = tf.nn.embedding_lookup(self.entity_embedding, self.iterator.t)
                self.r = tf.nn.embedding_lookup(self.relation_embedding, self.iterator.r)
                self.h_neg = tf.nn.embedding_lookup(self.entity_embedding, self.iterator.h_neg)
                self.t_neg = tf.nn.embedding_lookup(self.entity_embedding, self.iterator.t_neg)

            score_pos = self._score_func(self.h, self.r, self.t)
            score_neg = self._score_func(self.h_neg, self.r, self.t_neg)
            self.predict = score_pos
            self.loss = tf.reduce_sum(tf.maximum(0.0, self.params.margin + score_pos - score_neg),
                                      name='max_margin_loss')
            tf.summary.scalar(name=self.loss.op.name, data=self.loss, step=self.global_step)
            optimizer = get_optimizer_instance(self.params.optimizer, self.params.learning_rate)
            self.global_step = tf.Variable(initial_value=0, trainable=False, name='global_step')
            self.train_op = optimizer.minimize(self.loss, global_step=self.global_step)
            self.merge =
        self._model_stats()  # print model statistics info

    def _score_func(self, h, r, t):
        """Score / energy functions f_r(h, t), must be implemented in subclasses.
            f_r(h,t) = |Mr1*h+r-Mr2*t| constraints on the norm <=1
        Args:
            h, r, t: Tensor (b, k)
        Returns:
            Score tensor (b, 1)
        """
        # Projection matrix Mr, shape (k, k), initialize with identity matrix.
        self.Mr1 = tf.compat.v1.get_variable("Mr1", [self.k, self.k], initializer=tf.initializers.identity(gain=0.1))
        self.Mr1 = tf.tile(tf.expand_dims(self.Mr1, 0), [self.b, 1, 1])  # (b, k, k)
        self.Mr2 = tf.compat.v1.get_variable("Mr2", [self.k, self.k], initializer=tf.initializers.identity(gain=0.1))
        self.Mr2 = tf.tile(tf.expand_dims(self.Mr2, 0), [self.b, 1, 1])  # (b, k, k)

        h = tf.expand_dims(h, axis=2)  # (b, k) -> (b, k, 1)
        t = tf.expand_dims(t, axis=2)  # (b, k) -> (b, k, 1)
        dis = tf.squeeze(tf.matmul(self.Mr1, h), axis=2) + r + tf.squeeze(tf.matmul(self.Mr2, t), axis=2)
        score = None
        if self.params.score_func.lower() == 'l1':  # L1 score
            score = tf.reduce_sum(tf.abs(dis), axis=1)
        elif self.params.score_func.lower() == 'l2':  # L2 score
            score = tf.sqrt(tf.reduce_sum(tf.square(dis), axis=1))

        return score


    @staticmethod
    def _model_stats():
        """Print trainable variables and total model size."""

        def reduce(function, iterable, initializer=None):
            it = iter(iterable)
            if initializer is None:
                value = next(it)
            else:
                value = initializer
            for element in it:
                value = function(value, element)
            return value

        def size(v):
            return reduce(lambda x, y: x * y, v.get_shape().as_list())


        print("Trainable variables")
        for v in tf.compat.v1.trainable_variables():
            print("  %s, %s, %s, %s" % (v.name, v.device, str(v.get_shape()), size(v)))
        print("Total model size: %d" % (sum(size(v) for v in tf.compat.v1.trainable_variables())))

    def train(self, sess):
        return sess.run([self.loss, self.train_op, self.merge])

    def eval(self, sess):
        return sess.run([self.loss, self.pred])

    def predict(self, sess):
        return sess.run([self.pred])

    def save(self, sess, path):
        saver = tf.compat.v1.train.Saver()
        saver.save(sess, path, global_step=self.global_step.eval())

    def train():
    # Training
    with tf.Graph().as_default() as sess:
        sess.run()
        tf.Graph()
        writer = tf.summary.create_file_writer(logdir="summary", sess.graph)  # graph

        for epoch in range(FLAGS.max_epoch):
            sess.run(iterator.initializer)
            model.train(sess)
            if not os.path.exists(FLAGS.model_dir):
                os.mkdir(FLAGS.model_dir)
            save_path = os.path.join(FLAGS.model_dir, "model.ckpt")
            model.save(sess, save_path)

            print('-----Start training-----')
            epoch_loss = 0.0
            step = 0
            while True:
                try:
                    batch_loss, _, summary = model.train(sess)
                    epoch_loss += batch_loss
                    step += 1
                    writer.add_summary(summary)
                except tf.errors.OutOfRangeError:
                    print('-----Finish training an epoch avg epoch loss={}-----'.format(epoch_loss / step))
                    break
                # show train batch metrics
                if step % FLAGS.stats_per_steps == 0:
                    time_str = datetime.datetime.now().isoformat()
                    print('{}\tepoch {:2d}\tstep {:3d}\ttrain loss={:.6f}'.format(
                        time_str, epoch + 1, step, batch_loss))

            if (epoch+1) % FLAGS.save_per_epochs == 0:
                if not os.path.exists(FLAGS.model_dir):
                    os.mkdir(FLAGS.model_dir)
                save_path = os.path.join(FLAGS.model_dir, "model.ckpt")
                model.save(sess, save_path)
                print("Epoch {}, saved checkpoint to {}".format(epoch+1, save_path))

In [ ]:
def load_vocab(vocab_file):
    """load vocab from vocab file.
    Args:
        vocab_file: vocab file path
    Returns:
        vocab_table, vocab, vocab_size
    """
    vocab_table = tf.contrib.lookup.index_table_from_file(
        vocabulary_file=vocab_file, default_value=0)
    vocab = []
    with codecs.getreader("utf-8")(tf.gfile.GFile(vocab_file, "rb")) as f:
        vocab_size = 0
        for word in f:
            vocab_size += 1
            vocab.append(word.strip())
    return vocab_table, vocab, vocab_size

In [ ]:
def load_model(sess, ckpt):
    with sess.as_default():
        with sess.graph.as_default():
            init_ops = [tf.global_variables_initializer(),
                        tf.local_variables_initializer(), tf.tables_initializer()]
            sess.run(init_ops)
            # load saved model
            ckpt_path = tf.train.latest_checkpoint(ckpt)
            if ckpt_path:
                print("Loading saved model: " + ckpt_path)
            else:
                raise ValueError("No checkpoint found in {}".format(ckpt))
            # reader = tf.train.NewCheckpointReader(ckpt+'model.ckpt_0.876-580500')
            # variables = reader.get_variable_to_shape_map()
            # for v in variables:
            #     print(v)
            saver = tf.train.Saver()
            saver.restore(sess, ckpt_path)

In [ ]:
def print_checkpoint(save_path):
    reader = tf.train.load_checkpoint(save_path)
    shapes = reader.get_variable_to_shape_map()
    dtypes = reader.get_variable_to_dtype_map()
    print(f"Checkpoint at '{save_path}':")
    for key in shapes:
        print(f"  (key='{key}', shape={shapes[key]}, dtype={dtypes[key].name}, "
              f"value={reader.get_tensor(key)})")